### Install/Update nessaccay dependancies for model

In [1]:
!pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --extra-index-url https://download.pytorch.org/whl/cu126
!pip install diffusers transformers accelerate -U

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu126


Import Ltx pipeline from hugging face to the hf_cache directory located in content

In [2]:
import torch
from diffusers import LTXPipeline, AutoModel
import os
os.environ["HF_HOME"] = "/content/hf_cache"

# fp8 layerwise weight-casting
transformer = AutoModel.from_pretrained(
    "Lightricks/LTX-Video",
    subfolder="transformer",
    torch_dtype=torch.float16
)
transformer.enable_layerwise_casting(
    storage_dtype=torch.float8_e4m3fn, compute_dtype=torch.float16
)

pipeline = LTXPipeline.from_pretrained("Lightricks/LTX-Video", cache_dir="/content/hf_cache", transformer=transformer, torch_dtype=torch.float16, low_cpu_mem_usage=True, use_safetensors=True)
pipeline.to("cuda")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from diffusers.utils import export_to_video
import gc
import time
from google.colab import files

def detect_file_changes(file_path, interval=1):
    last_modified = os.path.getmtime(file_path)
    while True:
        current_modified = os.path.getmtime(file_path)
        if current_modified != last_modified:
            print("File has changed!")
            last_modified = current_modified
            !gdown --id "file id" -O /content/file.txt
            break
        time.sleep(interval)

while True:
  gc.collect()
  torch.cuda.empty_cache()

  detect_file_changes("/content/drive/MyDrive/4th Year project read Folder/ReadFile.txt")

  file = open("/content/file.txt", "r")
  file_content = file.read()

  if file_content != 'n':
    video = pipeline  (
        prompt=file_content,
        negative_prompt="low resolution, inconsistent motion, blurry, jittery, distorted",
        width=384,
        height=256,
        num_frames=100,
        num_inference_steps=50,
    ).frames[0]
    export_to_video(video, "output.mp4", fps=10)
  else:
    break;
files.download("output.mp4")

File has changed!
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Whi9DW5GSOxNNc1ts1negwxyniVXjTDi
To: /content/file.txt
100% 367/367 [00:00<00:00, 1.60MB/s]


  0%|          | 0/50 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 148.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 146.12 MiB is free. Process 84600 has 14.60 GiB memory in use. Of the allocated memory 14.28 GiB is allocated by PyTorch, and 190.19 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [5]:
from google.colab import files

files.download("output.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Debuing Section (ignore)

In [ ]:
import time

def detect_file_changes(file_path, interval=1):
    last_modified = os.path.getmtime(file_path)
    while True:
        current_modified = os.path.getmtime(file_path)
        if current_modified != last_modified:
            print("File has changed!")
            last_modified = current_modified
            !gdown --id "1Whi9DW5GSOxNNc1ts1negwxyniVXjTDi" -O /content/file.txt
            break
        time.sleep(interval)

detect_file_changes("/content/drive/MyDrive/4th Year project read Folder/ReadFile.txt")

file = open("/content/file.txt", "r")
file_content = file.read()
if file_content != 'n':
  print("continue")
  print(repr(file_content))
else:
  print("stopping")

File has changed!
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Whi9DW5GSOxNNc1ts1negwxyniVXjTDi
To: /content/file.txt
100% 1.00/1.00 [00:00<00:00, 4.67kB/s]
stopping


In [ ]:
!du -sh /content/new_cache.zip


4.0K	/content/new_cache.zip


In [ ]:
!zip -r -s 5000m /content/hf_cache_split.zip /content/hf_cache


  adding: content/hf_cache/ (stored 0%)
  adding: content/hf_cache/xet/ (stored 0%)
  adding: content/hf_cache/xet/logs/ (stored 0%)
  adding: content/hf_cache/xet/logs/xet_20251112T111959788+0000_166.log (deflated 97%)
  adding: content/hf_cache/xet/https___cas_serv-tGqkUaZf_CBPHQ6h/ (stored 0%)
  adding: content/hf_cache/xet/https___cas_serv-tGqkUaZf_CBPHQ6h/staging/ (stored 0%)
  adding: content/hf_cache/models--Lightricks--LTX-Video/ (stored 0%)
  adding: content/hf_cache/models--Lightricks--LTX-Video/refs/ (stored 0%)
  adding: content/hf_cache/models--Lightricks--LTX-Video/refs/main (deflated 3%)
  adding: content/hf_cache/models--Lightricks--LTX-Video/blobs/ (stored 0%)
  adding: content/hf_cache/models--Lightricks--LTX-Video/blobs/7a68b2c8c080696a10109612a649bc69330991ecfea65930ccfdfbdb011f2686


zip error: Interrupted (aborting)


In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


In [ ]:
!pip uninstall -y sympy
!pip install sympy


Found existing installation: sympy 1.13.3
Uninstalling sympy-1.13.3:
  Successfully uninstalled sympy-1.13.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.8 MB/s eta 0:00:00


In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0))

x = torch.randn(2, 2).to("cuda")
print(x.device)

CUDA available: True
CUDA device: Tesla T4
cuda:0


In [ ]:
!git clone https://github.com/Lightricks/LTX-Video.git

Cloning into 'LTX-Video'...
remote: Enumerating objects: 481, done.
remote: Counting objects: 100% (293/293), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 481 (delta 204), reused 122 (delta 122), pack-reused 188 (from 2)
Receiving objects: 100% (481/481), 217.13 KiB | 1.56 MiB/s, done.
Resolving deltas: 100% (254/254), done.
Filtering content: 100% (25/25), 353.90 MiB | 42.37 MiB/s, done.


In [ ]:
%cd LTX-Video

!pip install -e .\[inference-script\]

/content/LTX-Video
Obtaining file:///content/LTX-Video
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 114.8 MB/s eta 0:00:00
  Building editable for ltx-video (pyproject.toml) ... done
  Created wheel for ltx-video: filename=ltx_video-0.1.2-0.editable-py3-none-any.whl size=14467 sha256=ebc61da6885e50d773ae5b82d9437a1061c8853dd24575978da666319840a47a
  Stored in directory: /tmp/pip-ephem-wheel-cache-bqkxs5x9/wheels/69/50/3c/3cfe534943dc41f29ecce6bca834dce7551ddc98a213aabfba
Successfully built ltx-video
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: tr

In [ ]:
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 25.0 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
!pip install packaging wheel ninja setuptools
!pip install --no-build-isolation git+https://github.com/Lightricks/LTX-Video-Q8-Kernels.git


Looking in indexes: https://download.pytorch.org/whl/cu128
  Cloning https://github.com/Lightricks/LTX-Video-Q8-Kernels.git to /tmp/pip-req-build-4i4qzuz4
  Running command git clone --filter=blob:none --quiet https://github.com/Lightricks/LTX-Video-Q8-Kernels.git /tmp/pip-req-build-4i4qzuz4
  Resolved https://github.com/Lightricks/LTX-Video-Q8-Kernels.git to commit f3066edea210082799ca5a2bbf9ef0321c5dd8fc
  Running command git submodule update --init --recursive -q
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
prompt = "The ancient castle stood atop a jagged cliff, its silhouette carved sharply against the stormy night sky. Towering spires pierced the dark clouds, their stone turrets slick and glistening under the relentless sheets of rain. Lightning crackled in the distance, briefly illuminating the sprawling fortress with a ghostly, silver light."

!python inference.py --prompt prompt --height 480 --width 704 --num_frames 10 --frame_rate 5 --output_path output.mp4 --pipeline_config configs/ltxv-2b-0.9.6-distilled.yaml --offload_to_cpu True

2025-10-23 14:13:46.359189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761228826.386663   28934 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761228826.395541   28934 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761228826.439591   28934 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761228826.439623   28934 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761228826.439628   28934 computation_placer.cc:177] computation placer alr

In [ ]:
!nvidia-smi


Thu Oct 23 13:39:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8              8W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----